In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [12]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [13]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [14]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [16]:
#będziemy poprawiać dane. Najpierw brand z małej litery
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [18]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
18275    [{"key":"Style","value":["Desert Boots"]},{"ke...
18276    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
18277    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18278    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18279    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 18280, dtype: object

In [19]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [20]:
test = { 'key': 'value'}
test['key']

'value'

In [21]:
str(test)

"{'key': 'value'}"

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [24]:
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [25]:
literal_eval(str_dict)[0]

{'key': 'Gender', 'value': ['Men']}

In [28]:
literal_eval(str_dict)[0]['value']

['Men']

In [29]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"','"'))

df['features_parsed'] = df['features'].map(parse_features)

In [32]:
df['features_parsed'].head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict



df['features_parsed'] = df['features'].map(parse_features)

In [35]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [36]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()))
len(keys)

476

In [37]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [38]:
#dołożyliśmy kolumny feat_
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_stone type', 'feat_expandable', 'feat_style code',
       'feat_watch shape', 'feat_amazonbestsellersrank',
       'feat_diamond clarity', 'feat_dial markers', 'feat_authenticity',
       'feat_chest size', 'feat_number of compartments'],
      dtype='object', length=526)

In [39]:
df[ df['feat_athlete'].isnull()].shape

(18272, 526)

In [40]:
df.shape[0]

18280

In [41]:
df[ False == df['feat_athlete'].isnull()].shape[0]

8

In [44]:
df[ False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100 #ile procent nie pustych

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [46]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [47]:
{k: v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]


In [49]:
df[ df.brand != df.feat_brand].shape #ile razy oryginalny brand jest różny od tego pozyskanego z kolumny features

(18228, 531)

In [51]:
df[ df.brand != df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [53]:
df[ df.brand != df.feat_brand].shape

(9434, 531)

In [54]:
df[ df.brand != df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
feats = ['']

In [55]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31400977316232, 4.165762447354219)

In [57]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat' ]
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats , model)

(-57.133570821960504, 4.304791054763255)

In [58]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [60]:
#usuwamy manufactuer_part_number i color, bo mało istotna
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat' ]
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats , model)

(-57.16633995921531, 4.190808053260283)

In [61]:
df['brand'].value_counts()

nike            1777
puma             609
ralph lauren     526
vans             386
new balance      371
                ... 
levy's             1
banuce             1
tallia             1
ubisoft            1
poc                1
Name: brand, Length: 1732, dtype: int64

In [63]:
df['brand'].value_counts(normalize=True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
levy's          0.000055
banuce          0.000055
tallia          0.000055
ubisoft         0.000055
poc             0.000055
Name: brand, Length: 1732, dtype: float64

In [64]:
df[ df['brand'] == 'nike'].head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_clasp type,feat_eyewear frame style,feat_chain included,feat_condition,feat_interior pockets,feat_nickel free,feat_toe type,feat_sub type,feat_lens socket width,feat_fits model,feat_shoe category,feat_temple size:,feat_guaranteed authentic,feat_navy blue,feat_main color,feat_polarized,feat_transactionid,feat_type,feat_features & fastening,feat_waist size,feat_characteristics,feat_quantity in set,feat_auto,feat_is orthopedic,feat_outer material,feat_stone type,feat_expandable,feat_style code,feat_watch shape,feat_amazonbestsellersrank,feat_diamond clarity,feat_dial markers,feat_authenticity,feat_chest size,feat_number of compartments,feat_brand_cat,feat_color_cat,feat_gender_cat,feat_manufacturer part number_cat,feat_material_cat
50,AVpfI-6OLJeJML433P81,"B00YTD3MZE,B00YTD3UGK,B00YTD494M,B00YTD3K2O",nike,"Sporting Goods,Team Sports,Soccer,Clothing, Sh...",Orange,NaN,2016-07-07T19:13:47Z,2016-07-27T22:06:14Z,"[{""dateSeen"":[""2016-06-20T14:17:03Z""],""sourceU...",NaN,NaN,"[{""key"":""Sport"",""value"":[""Soccer""]},{""key"":""Ma...",NaN,http://i.ebayimg.com/images/g/uSIAAOSwPv9XHHYW...,NaN,newnikemenshypervenompheloniifgsoccercleatmtlc...,NaN,749896,"[{""dateSeen"":[""2016-06-18T00:00:00Z""],""name"":""...",New Nike Men's Hypervenom Phelon Ii Fg Soccer ...,72.90,72.90,NaN,NaN,New without box,NaN,USD,2016-07-07T19:13:47Z,2016-06-18T00:00:00Z,NaN,False,NaN,NaN,NaN,FREE,NaN,NaN,http://www.ebay.com/itm/NEW-Mens-9-11-NIKE-Hyp...,NaN,NaN,...,NaN,NaN,NaN,new without box,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,orange,NaN,NaN,cleats,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
51,AVpfI-6OLJeJML433P81,"B00YTD3MZE,B00YTD3UGK,B00YTD494M,B00YTD3K2O",nike,"Sporting Goods,Team Sports,Soccer,Clothing, Sh...",Orange,NaN,2016-07-07T19:13:47Z,2016-07-27T22:06:14Z,"[{""dateSeen"":[""2016-06-20T14:17:03Z""],""sourceU...",NaN,NaN,"[{""key"":""Sport"",""value"":[""Soccer""]},{""key"":""Ma...",NaN,http://i.ebayimg.com/images/g/uSIAAOSwPv9XHHYW...,NaN,newnikemenshypervenompheloniifgsoccercleatmtlc...,NaN,749896,"[{""dateSeen"":[""2016-06-18T00:00:00Z""],""name"":""...",New Nike Men's Hypervenom Phelon Ii Fg Soccer ...,69.90,69.90,NaN,NaN,New without box,NaN,USD,2016-07-07T19:13:47Z,2016-06-18T00:00:00Z,NaN,False,773adam773adam - Ebay.com,NaN,NaN,FREE,NaN,NaN,http://www.ebay.com/itm/NEW-Mens-10-5-12-NIKE-...,NaN,NaN,...,NaN,NaN,NaN,new without box,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,orange,NaN,NaN,cleats,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
52,AVpfI-6OLJeJML433P81,"B00YTD3MZE,B00YTD3UGK,B00YTD494M,B00YTD3K2O",nike,"Sporting Goods,Team Sports,Soccer,Clothing, Sh...",Orange,NaN,2016-07-07T19:13:47Z,2016-07-27T22:06:14Z,"[{""dateSeen"":[""2016-06-20T14:17:03Z""],""sourceU...",NaN,NaN,"[{""key"":""Sport"",""value"":[""Soccer""]},{""key"":""Ma...",NaN,http://i.ebayimg.com/images/g/uSIAAOSwPv9XHHYW...,NaN,newnikemenshypervenompheloniifgsoccercleatmtlc...,NaN,749896,"[{""dateSeen"":[""2016-06-18T00:00:00Z""],""name"":""...",New Nike Men's Hypervenom Phelon Ii Fg Soccer ...,66.40,66.40,NaN,NaN,New without box,NaN,USD,2016-07-07T19:13:47Z,2016-06-18T00:00:00Z,NaN,True,773adam773adam - Ebay.com,NaN,NaN,FREE,NaN,NaN,http://www.ebay.com/itm/NEW-Mens-10-5-12-NIKE-...,NaN,NaN,...,NaN,NaN,NaN,new without box,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,orange,NaN,NaN,cleats,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,-1
54,AVpfA-RILJeJML430gFn,NaN,nike,"Clothing, Shoes 

In [65]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [67]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats , model)

(-57.10703213684318, 4.255662143416629)

In [68]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/white/gum light brown', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '705172010', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'type': 'cleats', 'condition': 'new with box'},
       {'gender': 'men', 'size': 'xl', 'brand': 'nike', 'fabric content': '100% cotton', 'color': 'red/black'},
       {}], dtype=object)

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [71]:
feats_cat =  [x for x in df.columns if '_cat' in x and x != 'feat_catalog']
feats_cat

['brand_cat',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_smart watch_cat',
 'feat_is weather-resistant_cat',
 'feat_power type_cat',
 'feat_dimensions_cat',
 'feat_metal stamp_cat',
 'feat_nike id_cat',
 'feat_fit to_cat',
 'feat_lens tint_cat',
 'feat_shipping weight_cat',
 'feat_age gender group_cat',
 'feat_weight_cat',
 'feat_genre_cat',
 'feat_theme_cat',
 'feat_fine or fashion_cat',
 'feat_parts_cat',
 'feat_date first available_cat',
 'feat_battery voltage_cat',
 'feat_watch case shape_cat',
 'feat_international shipping_cat',
 'feat_diameter_cat',
 'feat_wheeled_cat',
 'feat_atpv arc rating_cat',
 'feat_era_cat',
 'feat_fastener_cat',
 'feat_band color_cat',
 'feat_frame material:_cat',
 'feat_used with equipment type_cat',
 'feat_best sellers rank_cat',
 'feat_resolution_cat',
 'feat_diamond color_cat',
 'feat_colour code_cat',
 'feat_lining material_cat',
 '

In [72]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]
feats += feats_cat
feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats , model)

(-57.70299041387691, 4.283399699513642)

In [73]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2432 ± 0.0146,brand_cat
0.1018 ± 0.0074,feat_material_cat
0.0121 ± 0.0025,feat_brand_cat
0.0119 ± 0.0015,feat_weight_cat
0.0109 ± 0.0031,feat_adjustable_cat
0.0101 ± 0.0010,feat_fabric content_cat
0.0056 ± 0.0003,feat_shoe category_cat
0.0053 ± 0.0012,feat_color_cat
0.0046 ± 0.0012,feat_resizable_cat
0.0044 ± 0.0004,feat_fabric material_cat


In [81]:
feats = ['brand_cat', 'feat_material_cat', 'feat_brand_cat', 'feat_weight_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats , model)

(-57.01158358215705, 4.096843772430399)

In [82]:
feats = ['brand_cat', 'feat_material_cat', 'feat_brand_cat', 'feat_weight_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats , model)

X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.02825662670343, 4.050334211270081)


Weight,Feature
0.2784 ± 0.0089,brand_cat
0.1086 ± 0.0128,feat_material_cat
0.0357 ± 0.0020,feat_brand_cat
0.0118 ± 0.0008,feat_weight_cat


In [0]:
!git add matrix_one/day4.ipynb